<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       QQNorm function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'><b>QQNorm </b> Function determines if values in input data columns follow normal distribution or not. 
    It returns the quantiles of the column values and corresponding theoretical quantile
    values from a normal distribution. If the column values are normally distributed, then
    the quantiles of column values and normal quantile values appear in a straight line
    when plotted on a 2D graph.
         <br> In this notebook we will see how we can use the QQ_Norm function available in Vantage.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import *

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_QQNorm_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_TitanicSurvival_cloud');"        # Takes 30 seconds
#%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_TitanicSurvival_local');" 

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
tdf = DataFrame(in_schema("DEMO_TitanicSurvival", "Passenger_Data"))
print("Shape of the data: ", tdf.shape)
tdf

In [ ]:
tdf.tdtypes

<p style = 'font-size:16px;font-family:Arial'>Getting NULL percentage for all columns and imputing the values for NULLs</p>

In [ ]:
colsum = ColumnSummary(
    data  = tdf,
    target_columns = [':']
)
colsum.result
cs = colsum.result.filter(items = ['ColumnName', 'Datatype', 'NullPercentage', 'NullCount'])
cs[cs['NullPercentage'] > 0.0]

<p style = 'font-size:16px;font-family:Arial'>Imputing missing value of age with mean value.</p>

In [ ]:
si_fit_obj = SimpleImputeFit(data=tdf,
                              stats_columns="age",
                              stats="median"
                              )
si_fit_obj.output

In [ ]:
si_trans_obj = SimpleImputeTransform(data=tdf,
                                 object=si_fit_obj.output
                                 )
si_trans_obj.result

In [ ]:
titanic_si = si_trans_obj.result

In [ ]:
window_func_age= titanic_si.age.window(order_columns="age")

In [ ]:
rank_age = titanic_si.assign(rank_age=window_func_age.rank())
rank_age.select(['passenger','age','rank_age']).sort('rank_age')

In [ ]:
help(QQNorm)

In [ ]:
obj_QQ = QQNorm(data=rank_age,
                 target_columns=["age"],
                 rank_columns=["rank_age"])

In [ ]:
df_qq = obj_QQ.result
df_qq

In [ ]:
plot = df_qq.plot(x=df_qq.age,
                y=df_qq.age_theoretical_quantiles,
                title="age distribution",
               kind="scatter")
plot.show()

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. Cleanup</b>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_TitanicSurvival');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>QQNorm function reference: <a href = 'https://docs.teradata.com/search/all?query=TD_QQNorm&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>